In [ ]:
import sys
from pathlib import Path

# Go two levels up to reach project root
project_root = Path.cwd().parents[1]  # from subfolder/ to notebook_folder/ to project_root/
scripts_path = project_root / "scripts"

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [ ]:
from scripts.general_scripts.features_engineering import quicky_data, lag_features_indicators
from scripts.general_scripts.pipelines import price_model
from scripts.general_scripts.predict import future_price_prediction
from scripts.general_scripts.lstm import LSTMModelMultiStep, LSTMModelMultiOutput
from scripts.general_scripts.helper import CustomizedLoss
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
scaler = StandardScaler()
model_type = LSTMModelMultiOutput
criterion = CustomizedLoss()
n_lags = 5
n_forecasts = 2
seasonal_periods = 261

In [ ]:
# 📂 Load dataset
file_path_1 = "../../ready_data/vn_index_data/cleaned_vn_index_data.csv"
df = pd.read_csv(file_path_1)
print(df.columns)
df_1 = quicky_data(df.copy(), seasonal_periods, plot=True)

In [ ]:
df_1.tail(20)

In [ ]:
df

In [ ]:
for n_forecast in range(1,3):
    for i in range(5):
        df_2 = quicky_data(df.copy().iloc[:-n_forecast*(i+1)], seasonal_periods)
        
        baseline = ExponentialSmoothing(
        df_2['VN_Index_Close'],
        trend='add',            # additive trend
        seasonal='add',         # additive seasonality
        seasonal_periods=seasonal_periods
        ).fit()

        # 2) Forecast the next day (one‐step ahead):
        price_forecast = baseline.forecast(n_forecast)

        # 3) If you want to extract the trend/seasonal forecasts separately:
        fitted_components = baseline.fittedvalues  # this is price = level+trend+seasonal
        level = baseline.level                     # the “smoothed level” ≈ trend
        seasonal = baseline.season                 # the seasonal factors

        print(price_forecast)
                    
        data = df_2[['residual', 'trend', 'seasonal', 'dow_sin', 'dow_cos', 'month_sin', 'month_cos']]
        
        # 🚀 Train the model and get the test set
        model, X_test_tensor, scaler, y_pred = price_model(data, scaler, model_type, criterion, tuning=False, train_seq_len=n_lags, test_seq_len=n_forecast, seasonal_periods=seasonal_periods, epochs=50)
        future_price_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast, seasonal_periods=seasonal_periods)

In [ ]:
data = df_1[['residual', 'trend', 'seasonal', 'dow_sin', 'dow_cos', 'month_sin', 'month_cos']]
# 🚀 Train the model and get the test set
model, X_test_tensor, scaler, y_pred = price_model(data, scaler, model_type, criterion, tuning=False, train_seq_len=n_lags, test_seq_len=n_forecast, 
                                                   seasonal_periods=seasonal_periods, epochs=50)
                                                   # params={'hidden_size': 64, 'num_layers': 2, 'dropout': 0.4304905019702554, 'learning_rate': 0.0022837602160213867, 'batch_size': 128})

# 🔮 Generate future predictions
future_price_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast, seasonal_periods=seasonal_periods)